In [ ]:
# Import necessary packages 

import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
from scipy.optimize import curve_fit
import numpy as np
from astropy.modeling import models
import astropy.units as u


from os import path

import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits

from ppxf.ppxf import ppxf
import ppxf.ppxf_util as util
import ppxf.miles_util as lib

ppxf_dir = path.dirname(path.realpath(lib.__file__))

In [ ]:
# Load in a data file (spectra)
hdul = fits.open("spec-1645-53172-0251.fits")

for i in range(len(hdul)):
    table_data = hdul[i].data
    #print("Table", i+1)
    #print(table_data)
table = Table(hdul[1].data)

# Define our parameters: wavelenght, flux, and the model spectra
model = table["model"]
loglam = table["loglam"]
wave = 10**loglam
flux = table["flux"]

galaxy = flux/np.median(flux)

#### From ppxf page: 

The SDSS wavelengths are in vacuum, while the MILES ones are in air. For a rigorous treatment, the SDSS vacuum wavelengths should be converted into air wavelengths and the spectra should be resampled. To avoid resampling, given that the wavelength dependence of the correction is very weak, it can be approximated with a constant factor.

In [ ]:
# Input source redshift 
z=0.0423

wave *= np.median(util.vac_to_air(wave)/wave)
rms = 0.019  # rms scatter of the spectrum residuals
goodpixels = np.arange(galaxy.size)  # fit full spectrum
noise = np.full_like(galaxy, rms)
wave_good = wave[goodpixels]

lam_range_gal = np.array([np.min(wave_good), np.max(wave_good)])

In [ ]:
# Read in photometry file

phot_file = Table("photometry.fits")

phot_galaxy = np.array(phot_file["flux"]) 
p_noise = np.array(phot_file["flux_err"])
phot_noise = p_noise/phot_galaxy

# Define the photometry bands used

bands = ['SDSS/u', 'SDSS/g', 'SDSS/r', 'SDSS/i', 'SDSS/z']
phot_lam, phot_templates, ok_temp = util.synthetic_photometry(templates, miles.lam_temp, bands, redshift=z, quiet=1)
phot = {"templates": phot_templates, "galaxy": phot_galaxy, "noise": phot_noise, "lam": phot_lam}

In [ ]:
from astropy import constants as const

#velscale = c*Delta[ln(lam)]

c = 299792.458  # Speed of light in km/s
d_ln_lam = np.diff(np.log(wave_good))
velscale = c * np.mean(d_ln_lam)

FWHM_gal = 2.76  # SDSS has an approximate instrumental resolution FWHM of 2.76A.

In [ ]:
# Determine path to the directory where the package models are stored
pathname = ppxf_dir + '/miles_models/Eun1.30*.fits'
miles = lib.miles(pathname, velscale, FWHM_gal, norm_range=[5070, 5950])
reg_dim = miles.templates.shape[1:]

# Read in stellar and gas templates (the latter based on chosen emission lines)
stars_templates = miles.templates.reshape(miles.templates.shape[0], -1)

gas_templates, gas_names, line_wave = util.emission_lines(miles.ln_lam_temp, lam_range_gal, FWHM_gal)

templates = np.hstack([stars_templates, gas_templates])

In [ ]:
vel = c*np.log(1 + z)   # eq.(8) of Cappellari (2017)
start = [vel, 60]     # (km/s), starting guess for [V, sigma]

n_stars = stars_templates.shape[1]
n_gas = len(gas_names)
component = [0]*n_stars + [1]*n_gas
gas_component = np.array(component) > 0  # gas_component=True for gas templates

moments = [2, 2] #[V, sigma]
# adopt the same starting value for both gas and stars

### Run ppxf!

In [ ]:
pp = ppxf(templates, galaxy, noise, velscale, start=start, 
          degree=-1, mdegree=8, lam=wave, lam_temp=miles.lam_temp, regul=1/rms,
          reg_dim=reg_dim, gas_component=gas_component, reddening=0, 
          gas_names=gas_names, goodpixels=goodpixels, phot=phot)

### Optional: plot light fraction and speactra fits

In [ ]:
light_weights = pp.weights[~gas_component]      # Exclude weights of the gas templates
light_weights = light_weights.reshape(reg_dim)  # Reshape to (n_ages, n_metal)
light_weights /= light_weights.sum()            # Normalize to light fractions

miles.mean_age_metal(light_weights);

mass_weights = light_weights/miles.flux
mass_weights /= mass_weights.sum()              # Normalize to mass fractions
miles.mass_to_light(mass_weights, band="r");


plt.figure(figsize=(10,5))
plt.subplot(211)
pp.plot(spec=True, phot=False, gas_clip=True)
plt.subplot(212)
pp.plot(spec=False, phot=True, gas_clip=True)
plt.tight_layout()

plt.figure(figsize=(10,3))
miles.plot(light_weights)
plt.title("Light Fraction");